# Tarefa: Tuning regressão carros usados

## Etapa 1: Importação das bibliotecas

In [1]:
!pip install skorch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.7/193.7 KB 1.7 MB/s eta 0:00:00


In [2]:
import pandas as pd
import torch.nn as nn        
from skorch import NeuralNetRegressor
import torch
from sklearn.model_selection import GridSearchCV
torch.__version__

'1.13.1+cu116'

## Etapa 2: Base de dados

In [3]:
torch.manual_seed(123)

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
base = pd.read_csv('/content/drive/MyDrive/Deep Learing de A à Z com PyTorch/Bases/autos.csv', encoding = 'ISO-8859-1')
base = base.drop('dateCrawled', axis = 1)
base = base.drop('dateCreated', axis = 1)
base = base.drop('nrOfPictures', axis = 1)
base = base.drop('postalCode', axis = 1)
base = base.drop('lastSeen', axis = 1)
base = base.drop('name', axis = 1)
base = base.drop('seller', axis = 1)
base = base.drop('offerType', axis = 1)

base = base[base.price > 10]
base = base.loc[base.price < 350000]

valores = {'vehicleType': 'limousine', 'gearbox': 'manuell',
           'model': 'golf', 'fuelType': 'benzin',
           'notRepairedDamage': 'nein'}
base = base.fillna(value = valores)

previsores = base.iloc[:, 1:13].values
preco_real = base.iloc[:, 0].values.reshape(-1, 1)

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
onehotencoder = ColumnTransformer(transformers = [("OneHot", OneHotEncoder(), [0, 1, 3, 5, 8, 9, 10])], remainder = 'passthrough')
previsores = onehotencoder.fit_transform(previsores).toarray()

previsores = previsores.astype('float32')
preco_real = preco_real.astype('float32')

## Etapa 3: Construção do modelo

In [6]:
class regressor_torch(nn.Module):
    def __init__(self):
        super().__init__()
        self.dense0 = nn.Linear(316, 158)
        torch.nn.init.uniform_(self.dense0.weight)
        self.dense1 = nn.Linear(158, 158)
        torch.nn.init.uniform_(self.dense1.weight)
        self.dense2 = nn.Linear(158, 1)
        
        self.activation = nn.ReLU()
        
    def forward(self, X):
        X = self.dense0(X)
        X = self.activation(X)
        X = self.dense1(X)
        X = self.activation(X)
        X = self.dense2(X)
        return X

In [7]:
regressor_sklearn = NeuralNetRegressor(module = regressor_torch, 
                                       optimizer = torch.optim.Adam,
                                       max_epochs = 100,
                                       batch_size = 300,
                                       train_split = False)

## Etapa 4: Tuning dos parâmetros

In [8]:
params = {'criterion': [torch.nn.MSELoss, torch.nn.L1Loss, torch.nn.SmoothL1Loss]}

In [9]:
grid_search = GridSearchCV(estimator = regressor_sklearn,
                           param_grid = params,                           
                           cv = 2)

In [10]:
grid_search = grid_search.fit(previsores, preco_real)

  epoch         train_loss     dur
-------  -----------------  ------
      1  342807653010.7596  4.3298
      2  93353525.5115  3.6699
      3  91178757.3371  5.0409
      4  89698859.8379  3.5790
      5  88085300.0689  3.5794
      6  1751414198.7798  4.9354
      7  2666851432.6642  3.7813
      8  1563079933.0009  3.5449
      9  64893098.1737  3.7554
     10  440145277.3294  4.9448
     11  250000365.3278  4.5946
     12  217514184.3989  3.6266
     13  155820011.4736  5.2126
     14  118724274.4567  3.6075
     15  84699271.3349  3.6165
     16  69533348.8235  4.8953
     17  57846926.9610  4.0315
     18  51754114.1029  3.7251
     19  44929968.4642  4.1695
     20  42653389.9590  5.1124
     21  41529418.9045  3.7772
     22  39392499.1709  3.8227
     23  38405489.5957  5.4156
     24  42781095.4957  3.8604
     25  39003705.7340  4.0064
     26  40433266.4324  5.5054
     27  43518949.7687  3.9569
     28  38055892.3114  3.9962
     29  43579886.5935  5.5408
     30  3828356

In [11]:
melhores_parametros = grid_search.best_params_
melhor_resultado = grid_search.best_score_